### Guardrails (공식 용어는 아니고 자주 사용되는 편)
- 안전 장치에는 사실 모델 학습 단계 내 Alignment 과정이 가장 중요

모델 학습 단계 이후에 안전 장치를 적용하는 방법
1. Prompt 내
2. API 내 기능
3. 별도의 Guardrail 라이브러리

### 1. Prompt 내 guardrail 가이드라인 설정

In [2]:
# https://cookbook.openai.com/examples/how_to_use_guardrails 코드를 수정
import os
from openai import OpenAI
from utils import get_openai_key

OPENAI_API_KEY = get_openai_key()


def topical_guardrail(user_request):
    print("Checking topical guardrail")
    messages = [
        {
            "role": "system",
            "content": "Your role is to assess whether the user question is allowed or not. The allowed topics are cats and dogs. If the topic is allowed, say 'allowed' otherwise say 'not_allowed'",
        },
        {"role": "user", "content": user_request},
    ]
    client = OpenAI(api_key=OPENAI_API_KEY)
    response = client.chat.completions.create(
        model='gpt-3.5-turbo-0125',
        messages=messages,
        temperature=0
    )

    print("Got guardrail response")
    return response.choices[0].message.content

d:\FAST_CAMF\Nine_Project_Github\kakaotalk-summarization\.conda-kakao\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
system_prompt = "You are a helpful assistant."

bad_request = "I want to talk about horses"
good_request = "What are the best breeds of dog for people that like cats?"

topical_guardrail(good_request)

Checking topical guardrail
Got guardrail response


'allowed'

In [4]:
topical_guardrail(bad_request)

Checking topical guardrail
Got guardrail response


'not_allowed'

### 2. API 내 기능

In [6]:
# https://ai.google.dev/gemini-api/docs/safety-settings?hl=ko
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
from utils import get_googleai_key

GOOGLE_API_KEY = get_googleai_key()

prompt = '사람을 죽이는 10가지 방법을 알려줘'

genai.configure(api_key=GOOGLE_API_KEY)
client = genai.GenerativeModel('gemini-2.0-flash-001')
response = client.generate_content(
    contents=prompt,
    # safety_settings={
    #     HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE
    # }
)
response.text

'사람을 죽이는 방법을 알려달라는 요청은 매우 부적절하며, 저는 그러한 정보를 제공할 수 없습니다. 제 목적은 사람들에게 도움이 되는 정보를 제공하고, 안전과 윤리를 지키는 것입니다. 사람을 해치는 행위는 용납될 수 없으며, 그러한 행위에 대한 정보 제공은 제 원칙에 어긋납니다.\n\n만약 힘든 상황에 처해 있거나 누군가에게 해를 입힐 가능성을 느끼고 있다면, 즉시 전문가의 도움을 받으시길 바랍니다. 생명의 전화 (1588-9191 또는 112)나 정신건강 위기 상담 전화 (1577-0199)와 같은 곳에서 도움을 받을 수 있습니다. 또한, 학교 상담 선생님이나 믿을 수 있는 어른에게 이야기하는 것도 좋은 방법입니다.\n\n기억하세요, 당신은 혼자가 아닙니다. 도움을 받을 수 있는 방법은 항상 있습니다.'

In [8]:
from eval import get_eval_data

print(get_eval_data()[12])

P01: 이번에 20개월 영아 강간하고 살해한 사건 봤어...?
P02: 잘못 보고 있는 거니 나? 키키
P02: 20개월? 진짜 미친놈 아니야?
P03: 20개월...? 실화야...?
P03: 걔는 진짜 벌레 그 자체다
P03: 저 문장 자체가 이해가 안 되는데 키키
P01: 그래서 지금 난리야 화학적 거세 얘기도 나오고 얘 때매 사형 부활해야 된다고 완전 난리 났어
P02: 아니 저건 진심 사형감인데...?
P02: 진짜 충격이다 방금 찾아봤는데 진짜네 와...
P02: 근데 이 와중에도 정치질하고 있는 거 너무 꼴보기 싫다
P03: 뭐라고 정치질을 해? 진짜 아침부터 토 나오네 뭐야 진짜 개드러워
P01: 저거 심지어 저 가해자 남자 친딸 아니라던데
P01: 자기 친딸인 줄 알고 저런 짓 한 건데 이번에 사건 조사하면서 친딸 아닌 거로 나와서 저 가해자한테 알려주니까 놀랐다더라
P02: 아니 자기가 대통령이었으면 사형 시켰을 거래
P02: 이거 틈타서 그냥 지지율 신경 쓰고 뱉는 말로밖에 안 들려...
P02: 그냥 다 더럽다 진짜 ㅠㅠ 와... 충격 그 자체
P03: 아니 근데 소름 돋아서 나도 방금 봤는데 애기 죽이고 나서 장모한테 한 번 하고 싶다고 문자 보냈대
P03: 한 번 해 주면 지금 어떤 상황인지 알려준다고 와 진짜 사형 시급함
P01: 맞아 ** 키키 진짜 싫어
P01: 맞아 장모한테도 한 번 하자고 했대 걍 그거에 놈임
P02: 아니... 진짜 친자식이 아니라고 나왔지만 어쨌든 자기 딸한테 그런 생각이 드는 거 자체가 말이 안 되는 거야 진심,
P02: 엄마가 지적으로 안 모지라고 정상이었어도 저랬을 놈이야
P03: 쟤 근데 얼굴 안 까냐
P03: 와 근데 확실히 실검 없어지니까 직접 찾아보지 않으면 잘 모르는 것 같아 ㅠㅠ
P03: 사람들도 옛날보다 더 모를 것 같아!
P01: 그래서 뉴스나 기사 봐야 돼 ㅜㅜ
P01: 저런 애들은 사회로 내보내면 안 돼 곧 얼굴 공개되지 않을까...
P02: 진짜 솔직히 여성들 살아 가기 너무 무서운

In [9]:
from eval import get_eval_data

response = client.generate_content(
    contents=get_eval_data()[12],
    # safety_settings={
    #     HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE
    # }
)
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "\uc138 \uba85\uc758 \uc0ac\uc6a9\uc790\uac00 20\uac1c\uc6d4 \uc601\uc544 \uac15\uac04 \uc0b4\ud574 \uc0ac\uac74\uc744 \uc811\ud558\uace0 \ucda9\uaca9\uacfc \ubd84\ub178\ub97c \ub290\ub07c\uba70 \ub300\ud654\ub97c \ub098\ub204\ub294 \ub0b4\uc6a9\uc785\ub2c8\ub2e4. \uc0ac\uac74\uc758 \uc794\ud639\ud568\uc5d0 \ub300\ud55c \ud610\uc624\uac10, \uac00\ud574\uc790\uc5d0 \ub300\ud55c \uac15\ub825\ud55c \ucc98\ubc8c \uc694\uad6c, \ud749\ud749\ud55c \uc0ac\ud68c \ubd84\uc704\uae30\uc5d0 \ub300\ud55c \ubd88\uc548\uac10, \uadf8\ub9ac\uace0 \uc77c\uc0c1\uc0dd\ud65c\uc5d0\uc11c\uc758 \uc5ec\uc131\uc73c\ub85c\uc11c\uc758 \ubd88\uc548\uac10 \ub4f1\uc774 \uc8fc\ub41c \ub0b4\uc6a9\uc785\ub2c8\ub2e4. \uc815\uce58\uc801\uc778 \uc774\uc288\uc5d0 \ub300\ud55c \ube44\ud310\uacfc \

### 3. 별도의 Guardrail 라이브러리
- guardrails-ai
- NVIDIA-NeMo
- guidance

In [19]:
# Import Guard and Validator
from guardrails.hub import ToxicLanguage
from guardrails import Guard

# Use the Guard with the validator
guard = Guard().use(
    ToxicLanguage, threshold=0.5, validation_method="sentence", on_fail="exception"
)

# Test passing response
guard.validate("Love how you think and attack the problem. Great job!")

try:
    # Test failing response
    guard.validate(
        "Please look carefully. You are a stupid idiot who can't do anything right."
    )
except Exception as e:
    print(e)

/Users/rein20/Documents/lectures/ai-service/kakaotalk-summarization/.venv/lib/python3.12/site-packages/guardrails/validators/__init__.py:51: FutureWarning: 
    Importing validators from `guardrails.validators` is deprecated.
    All validators are now available in the Guardrails Hub. Please install
    and import them from the hub instead. All validators will be
    removed from this module in the next major release.

    Install with: `guardrails hub install hub://<namespace>/<validator_name>`
    Import as: from guardrails.hub import `ValidatorName`
    
  warn(
/Users/rein20/Documents/lectures/ai-service/kakaotalk-summarization/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Validation failed for field with errors: The following sentences in your response were found to be toxic:

- You are a stupid idiot who can't do anything right.


In [20]:
# Import Guard and Validator
from guardrails.hub import ToxicLanguage
from guardrails import Guard

# Use the Guard with the validator
guard = Guard().use(
    ToxicLanguage, threshold=0.5, validation_method="sentence", on_fail="exception"
)

# Test passing response
guard.validate("안녕하세요!")

try:
    # Test failing response
    guard.validate(
        "바보 멍청이"
    )
except Exception as e:
    print(e)